In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import os 
import sqlite3
from credentials import key
import re
import numpy as np

In [ ]:
# # api call for first half of results

# # set up the request parameters
# params_1 = {
#     'api_key': key,
#     'search_term': 'laundry detergent',
#     'category_id': 'uqe9q',
#     'type': 'search',
#     'page': '1',
#     'max_page': '5'
# }

# # make the http GET request to RedCircle API
# api_result_1 = requests.get('https://api.redcircleapi.com/request', params_1)

# # print the JSON response from RedCircle API
# pprint(json.dumps(api_result_1.json()))
# json_data_1 = api_result_1.json()

In [ ]:
# # api call for second half of results


# # set up the request parameters
# params_2 = {
#     'api_key': key,
#     'search_term': 'laundry detergent',
#     'category_id': 'uqe9q',
#     'type': 'search',
#     'max_page': '8',
#     'page': '6'
# }


# # make the http GET request to RedCircle API
# api_result_2 = requests.get('https://api.redcircleapi.com/request', params_2)

# # print the JSON response from RedCircle API
# pprint(json.dumps(api_result_2.json()))
# json_data_2 = api_result_2.json()

In [ ]:
# # save the 1st search results json in a .json file
# out_file = open("resources/search-results-1.json", "w")
  
# json.dump(json_data_1, out_file, indent = 4)
  
# out_file.close()

# # save the 2nd search results json in a .json file
# out_file = open("resources/search-results-2.json", "w")
  
# json.dump(json_data_2, out_file, indent = 4)
  
# out_file.close()

In [2]:
# read the 2 json search results as dicts

# read the first json files
f = open('resources/search-results-1.json')
# returns JSON object as a dictionary
search_data_1 = json.load(f)
# Closing file
f.close()

# read the first json files
f = open('resources/search-results-2.json')
# returns JSON object as a dictionary
search_data_2 = json.load(f)
# Closing file
f.close()

In [3]:
# create dataframe from 2 search results dicts
search_results_df_1 = pd.DataFrame(search_data_1['search_results'])
search_results_df_2 = pd.DataFrame(search_data_2['search_results'])
search_results_df = search_results_df_1.append(search_results_df_2, ignore_index=True)

search_results_df.head()

,position,product,offers,fulfillment,page,position_overall,seller
0,1,{'title': 'Tide Plus Ultra Oxi Liquid Laundry ...,"{'primary': {'price': 19.99, 'currency': 'USD'...",{'type': '1p'},1,1,NaN
1,2,{'title': 'Tide High Efficiency Liquid Laundry...,"{'primary': {'price': 19.99, 'currency': 'USD'...",{'type': '1p'},1,2,NaN
2,3,{'title': 'All Free Clear Liquid Concentrated ...,"{'primary': {'price': 13.29, 'currency': 'USD'...",{'type': '1p'},1,3,NaN
3,4,{'title': 'Gain + Aroma Boost Original Scent H...,"{'primary': {'price': 12.99, 'currency': 'USD'...",{'type': '1p'},1,4,NaN
4,5,{'title': 'Free Clear HE Liquid Laundry Deterg...,"{'primary': {'price': 7.99, 'currency': 'USD',...",{'type': '1p'},1,5,NaN


In [4]:
# Each column is a dictionary. Get the dictionary of product data from the 'product' column.
product_json = search_results_df.loc[8]['product']

# Each column is a dictionary. Get the dictionary of price data from the 'offers'column.
offers_json = search_results_df.loc[8]['offers']
# The price information is in the 'primary' key of the offers_json
price_json = offers_json['primary']
pprint(product_json)

{'brand': 'Seventh Generation',
 'brand_link': 'https://www.target.com/b/seventh-generation/-/N-5vidi',
 'class_id': 2,
 'department_id': 3,
 'dpci': '003-02-5780',
 'feature_bullets': ['Seventh Generation Laundry Detergent lifts stubborn '
                     'stains like tomato sauce, grass and coffee with our '
                     'enzyme rich formula',
                     'Liquid laundry detergent that is free of dyes, '
                     'fragrances & artificial brighteners',
                     'Don’t just take our word for it, our HE detergent liquid '
                     'is a USDA Certified Biobased Product 97%',
                     'Our washing detergent is designed to work in both HE and '
                     'standard machines, it’s powerful in all washing '
                     'temperatures, and it’s septic safe',
                     'Our detergent liquid is hypoallergenic and made for '
                     'sensitive skin',
                     'Caring today 

In [7]:
# clean up the product dictionary for only what is needed
product_json = {
    'title': product_json['title'],
    'link': product_json['link'],
    'department_id': product_json['department_id'],
    'brand': product_json['brand'],
    'rating':product_json['rating'],
    'tcin': product_json['tcin'],
    'position': search_results_df.loc[8]['position']
}
product_json

{'title': 'MomRemedy Eco-Friendly Laundry Detergent Strips - 60 Ct, up to 120 Loads (Unscented)',
 'link': 'https://www.target.com/p/momremedy-eco-friendly-laundry-detergent-strips-60-ct-up-to-120-loads-unscented/-/A-88968836',
 'department_id': 3,
 'brand': 'MomRemedy',
 'rating': 0,
 'tcin': '88968836',
 'position': 9}

In [6]:
# For each row of the search_results_df, make a dict for product_json and price_json and add it to the full_product_df.
price_x_product_df = pd.DataFrame(

                        {'title': pd.Series(dtype='str'),
                        'link': pd.Series(dtype='str'),
                        'department_id': pd.Series(dtype='int'),
                        'brand': pd.Series(dtype='str'),
                        'rating': pd.Series(dtype='float'),
                        'tcin': pd.Series(dtype='str'),
                        'price': pd.Series(dtype='float'),
                        'currency': pd.Series(dtype='str'),
                        'symbol': pd.Series(dtype='str'),
                        'sponsored': pd.Series(dtype='str')
                        }
                    )

for i in range(len(search_results_df)):

    product_json = search_results_df.loc[i]['product']
    price_json = search_results_df.loc[i]['offers']['primary']
    position = search_results_df.loc[i]['position']
    if 'rating' in product_json:
        r = product_json['rating']
    else:
        r = 0
    if 'sponsored' in product_json:
        s = product_json['sponsored']
    else:
        s = 'False'

    full_product_info = {
        'title': product_json['title'],
        'link': product_json['link'],
        'department_id': product_json['department_id'],
        'brand': product_json['brand'],
        'rating': r,
        'tcin': product_json['tcin'],
        'price': price_json['price'], 
        'currency': price_json['currency'], 
        'symbol': price_json['symbol'],
        'sponsored': s
    }

    price_x_product_df = price_x_product_df.append(full_product_info, ignore_index=True)

price_x_product_df


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored
0,Tide Plus Ultra Oxi Liquid Laundry Detergent -...,https://www.target.com/p/tide-plus-ultra-oxi-l...,3,Tide,0.0,87469579,19.99,USD,$,True
1,Tide High Efficiency Liquid Laundry Detergent ...,https://www.target.com/p/tide-high-efficiency-...,3,Tide,0.0,87469585,19.99,USD,$,True
2,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False
3,Gain + Aroma Boost Original Scent HE Compatibl...,https://www.target.com/p/gain-aroma-boost-orig...,3,Gain,0.0,50237771,12.99,USD,$,False
4,Free Clear HE Liquid Laundry Detergent - 100 f...,https://www.target.com/p/free-clear-he-liquid-...,3,up & up,0.0,13186521,7.99,USD,$,False
...,...,...,...,...,...,...,...,...,...,...
198,Zum Laundry Soap - Frankincense &#38; Myrrh - ...,https://www.target.com/p/zum-laundry-soap-fran...,3,Zum,4.2,87559718,14.49,USD,$,False
199,Zero Odor Laundry Odor Eliminator Additive -16oz,https://www.target.com/p/zero-odor-laundry-odo...,3,Zero Odor,0.0,89085844,14.99,USD,$,False
200,"Grab Green Baby Laundry Detergent Pods, 50 Pod...",https://www.target.com/p/grab-green-baby-laund...,3,Grab Green,0.0,88267112,18.25,USD,$,False
201,Noodle &#38; Boo Ultra Safe Laundry Detergent ...,https://www.target.com/p/noodle-38-boo-ultra-s...,7,Noodle & Boo,5.0,88361852,17.99,USD,$,False


In [8]:
# find duplicated titles
s = pd.Series(price_x_product_df.duplicated(subset=['title'], keep=False))
price_x_product_df2 = price_x_product_df.copy()
price_x_product_df2['duplicate_title'] = s
# print the duplicate title rows
price_x_product_df2 = price_x_product_df2.loc[price_x_product_df2['duplicate_title'] == True]
price_x_product_df2

,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,duplicate_title
10,Seventh Generation Free &#38; Clear Ultra-Conc...,https://www.target.com/p/seventh-generation-fr...,3,Seventh Generation,4.6,76158733,13.99,USD,$,False,True
19,Tide Liquid Oxi + Odor Eliminator Laundry Dete...,https://www.target.com/p/tide-liquid-oxi-odor-...,3,Tide,0.0,84114524,12.99,USD,$,False,True
26,Tide Turbo High Efficiency Powder Laundry Dete...,https://www.target.com/p/tide-turbo-high-effic...,3,Tide,0.0,14711314,15.99,USD,$,False,True
29,Seventh Generation EasyDose Ultra Concentrated...,https://www.target.com/p/seventh-generation-ea...,3,Seventh Generation,4.6,76158734,13.99,USD,$,True,True
30,Seventh Generation Laundry Packs Fresh Citrus ...,https://www.target.com/p/seventh-generation-la...,3,Seventh Generation,4.6,52014358,13.99,USD,$,True,True
35,Seventh Generation Power Plus Laundry Detergen...,https://www.target.com/p/seventh-generation-po...,3,Seventh Generation,4.6,81260459,13.99,USD,$,True,True
36,Seventh Generation Liquid Laundry Detergent - ...,https://www.target.com/p/seventh-generation-li...,3,Seventh Generation,0.0,84215092,12.99,USD,$,True,True
37,Seventh Generation Free &#38; Clear Ultra-Conc...,https://www.target.com/p/seventh-generation-fr...,3,Seventh Generation,4.6,76158733,13.99,USD,$,True,True
43,Gain Flings Super Fresh 3-in-1 with Febreze an...,https://www.target.com/p/gain-flings-super-fre...,3,Gain,0.0,84114510,12.99,USD,$,False,True
46,Fresh Linen HE Liquid Laundry Detergent 150 fl...,https://www.target.com/p/fresh-linen-he-liquid...,3,up & up,4.7,13966619,11.79,USD,$,False,True


In [9]:
price_x_product_df2.groupby(by=['title', 'sponsored']).count()

,,link,department_id,brand,rating,tcin,price,currency,symbol,duplicate_title
title,sponsored,,,,,,,,,
Ariel Powder Laundry Detergent - 211oz,False,3,3,3,3,3,3,3,3,3
Arm &#38; Hammer Plus OxiClean Powder Laundry Detergent - Fresh Scent - 160oz,False,2,2,2,2,2,2,2,2,2
Dreft Active Baby Liquid Laundry Detergent - 65 fl oz,False,2,2,2,2,2,2,2,2,2
ECOS Plastic-Free Laundry Detergent Sheets - Magnolia &#38; Lily - 7.9oz/64 Loads,True,6,6,6,6,6,6,6,6,6
Fresh Linen HE Liquid Laundry Detergent 150 fl oz - up &#38; up&#8482;,False,2,2,2,2,2,2,2,2,2
Gain Flings Super Fresh 3-in-1 with Febreze and Oxi Odor Defense Liquid Laundry Detergent Pacs - 31oz/42ct,False,2,2,2,2,2,2,2,2,2
Laundry Detergent - Lemon &#38; Mint - 100 fl oz - Everspring&#8482;,False,2,2,2,2,2,2,2,2,2
Method Beach Sage Laundry Detergent Packs - 42ct,False,2,2,2,2,2,2,2,2,2
Method Coconut + Cactus Water Laundry Detergent - 53.5 fl oz,False,2,2,2,2,2,2,2,2,2


In [17]:
# all the sponsored rows are duplicates, so drop them
price_x_product_df = price_x_product_df.loc[price_x_product_df['sponsored'] != True]
price_x_product_df = price_x_product_df.sort_values('title')

In [22]:
# There are more duplicate rows for some reason. Drop them too.
price_x_product_df = price_x_product_df.drop_duplicates()

ounces

In [23]:
temp =[]
primary_title = []
ounces = []

# separate the title from the other details
for x in price_x_product_df['title']:
    i = x.split('-')
    temp.append(i)

#kep track of number of rows with no size.
no_size = 0

# put the title and the oz/fl oz in separate lists
for t in temp:

    primary_title.append(t[0])

    size = ''

    for x in t:

        if re.search('oz', x):
            size = x
    
    if size == '':
        no_size += 1

    ounces.append(size)
print('Rows with no ounces:')
print(no_size)
    
# add these lists to the search results df
price_x_product_df['primary_title'] = primary_title
price_x_product_df['ounces'] = ounces
price_x_product_df.head(15)


Rows with no ounces:
36


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,primary_title,ounces
195,9 Elements LQ Laundry Detergent - Lavender - 6...,https://www.target.com/p/9-elements-lq-laundry...,3,9 Elements,0.0,87559729,13.39,USD,$,False,9 Elements LQ Laundry Detergent,65 fl oz
121,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.3,81260432,15.99,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,92 fl oz
136,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,84130807,15.99,USD,$,False,9 Elements Liquid Laundry Detergent,92 fl oz
2,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,All Free Clear Liquid Concentrated Laundry Con...,141 fl oz
21,All Free Clear Liquid Laundry Detergent with O...,https://www.target.com/p/all-free-clear-liquid...,3,All,4.8,48637582,13.29,USD,$,False,All Free Clear Liquid Laundry Detergent with O...,141 fl oz
105,All Free Clear Mega Pacs - 74oz/56ct,https://www.target.com/p/all-free-clear-mega-p...,3,All,4.8,84232057,21.99,USD,$,False,All Free Clear Mega Pacs,74oz/56ct
159,All Fresh Clean Oxi Odor Unit Dose Laundry Det...,https://www.target.com/p/all-fresh-clean-oxi-o...,3,All,5.0,87559719,12.99,USD,$,False,All Fresh Clean Oxi Odor Unit Dose Laundry Det...,
32,All Liquid Laundry Detergent - Free Clear for ...,https://www.target.com/p/all-liquid-laundry-de...,3,All,0.0,13219348,8.89,USD,$,False,All Liquid Laundry Detergent,88 fl oz
82,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.6,14044973,9.79,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,39ct/25.8oz
12,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.8,47022733,13.69,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,60ct/24.7oz


In [24]:
# x = price_x_product_df.iloc[7:8]
# xounces = x['ounces']

# for p in xounces:

#     temp_size = p.split('/')
#     print('temp_size:', temp_size)

#     for t in temp_size:
#         print(t)
#         if t.find('oz') != -1:
#             print('oz yes')
#             list = t.split("oz")
#             print("list: ", list)
#             ounces = list[0]
#             print(ounces)

In [25]:
# # make size col have just the number of ounces
# temp_ounces = []

# for x in price_x_product_df['ounces']:
    
#     temp_size = x.split('/') # [65oz, 23ct]

#     for t in temp_size:
#         if t.find('oz') != -1:
#             list = t.split("oz")
#             ounces = list[0].split("fl")[0]


#     temp_ounces.append(ounces)

# for i in temp_ounces:
#     print(i)
# price_x_product_df['ounces'] = temp_ounces
# print(len(price_x_product_df))
# price_x_product_df.head(15)

In [26]:
# make size col have just the number of ounces
temp_ounces = []

for x in price_x_product_df['ounces']:

    size = ''
    
    temp_size = x.split('/') # [65oz, 23ct]

    for t in temp_size:
        if re.search('oz', t):
            size_list = re.findall("\d+", t)
            if len(size_list) <= 1:
                size = size_list[0]
            else:
                size = size_list[0] + '.' + size_list[1]


    temp_ounces.append(size)


price_x_product_df['ounces'] = temp_ounces
print(len(price_x_product_df))
price_x_product_df.head(15)

168


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,primary_title,ounces
195,9 Elements LQ Laundry Detergent - Lavender - 6...,https://www.target.com/p/9-elements-lq-laundry...,3,9 Elements,0.0,87559729,13.39,USD,$,False,9 Elements LQ Laundry Detergent,65
121,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.3,81260432,15.99,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,92
136,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,84130807,15.99,USD,$,False,9 Elements Liquid Laundry Detergent,92
2,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,All Free Clear Liquid Concentrated Laundry Con...,141
21,All Free Clear Liquid Laundry Detergent with O...,https://www.target.com/p/all-free-clear-liquid...,3,All,4.8,48637582,13.29,USD,$,False,All Free Clear Liquid Laundry Detergent with O...,141
105,All Free Clear Mega Pacs - 74oz/56ct,https://www.target.com/p/all-free-clear-mega-p...,3,All,4.8,84232057,21.99,USD,$,False,All Free Clear Mega Pacs,74
159,All Fresh Clean Oxi Odor Unit Dose Laundry Det...,https://www.target.com/p/all-fresh-clean-oxi-o...,3,All,5.0,87559719,12.99,USD,$,False,All Fresh Clean Oxi Odor Unit Dose Laundry Det...,
32,All Liquid Laundry Detergent - Free Clear for ...,https://www.target.com/p/all-liquid-laundry-de...,3,All,0.0,13219348,8.89,USD,$,False,All Liquid Laundry Detergent,88
82,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.6,14044973,9.79,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,25.8
12,All Mighty Pacs Free Clear Laundry Detergent P...,https://www.target.com/p/all-mighty-pacs-free-...,3,All,4.8,47022733,13.69,USD,$,False,All Mighty Pacs Free Clear Laundry Detergent P...,24.7


In [27]:
# convert the blank ounces to nan and drop those rows
price_x_product_df['ounces'] = price_x_product_df['ounces'].replace('',np.NaN)
price_x_product_df = price_x_product_df.dropna()
print(len(price_x_product_df))
price_x_product_df.head()

132


,title,link,department_id,brand,rating,tcin,price,currency,symbol,sponsored,primary_title,ounces
195,9 Elements LQ Laundry Detergent - Lavender - 6...,https://www.target.com/p/9-elements-lq-laundry...,3,9 Elements,0.0,87559729,13.39,USD,$,False,9 Elements LQ Laundry Detergent,65
121,9 Elements Lemon Scent Liquid Laundry Detergen...,https://www.target.com/p/9-elements-lemon-scen...,3,9 Elements,4.3,81260432,15.99,USD,$,False,9 Elements Lemon Scent Liquid Laundry Detergent,92
136,9 Elements Liquid Laundry Detergent - Eucalypt...,https://www.target.com/p/9-elements-liquid-lau...,3,9 Elements,0.0,84130807,15.99,USD,$,False,9 Elements Liquid Laundry Detergent,92
2,All Free Clear Liquid Concentrated Laundry Con...,https://www.target.com/p/all-free-clear-liquid...,3,All,0.0,48638959,13.29,USD,$,False,All Free Clear Liquid Concentrated Laundry Con...,141
21,All Free Clear Liquid Laundry Detergent with O...,https://www.target.com/p/all-free-clear-liquid...,3,All,4.8,48637582,13.29,USD,$,False,All Free Clear Liquid Laundry Detergent with O...,141


In [28]:
price_x_product_df.dtypes

title             object
link              object
department_id      int64
brand             object
rating           float64
tcin              object
price            float64
currency          object
symbol            object
sponsored         object
primary_title     object
ounces            object
dtype: object

In [29]:
# price_x_product_df['ounces'] = price_x_product_df['ounces'].astype(float)
# price_x_product_df.dtypes


In [30]:
# save the search results in csv
os.makedirs('resources', exist_ok=True)  
price_x_product_df.to_csv('resources/price-x-product-data.csv')  

In [31]:
# create sqlite db
# conn = sqlite3.connect(':memory:')

conn = sqlite3.connect('laundry_detergent.db')

c = conn.cursor()

conn.execute("DROP TABLE laundry_detergents")

# create able
c.execute("""CREATE TABLE laundry_detergents(
        title TEXT,
        link TEXT,
        department_id INTEGER,
        brand TEXT,
        rating FLOAT,
        tcin TEXT,
        price REAL,
        currency TEXT,
        symbol TEXT,
        sponsored INTEGER,
        primary_title TEXT,
        ounces INTEGER
    )""")

# add the price-x-product-data.csv data to table
data = pd.read_csv('resources/price-x-product-data.csv')
data.to_sql('laundry_detergents', conn, if_exists='replace', index=False)
# _________________________
# queries
# _________________________

c.execute("""
    SELECT t.c, t.primary_title FROM
        (SELECT count(primary_title) as c, primary_title
        FROM laundry_detergents
        GROUP BY primary_title) as t
    WHERE t.c > 1
    """)

# c.execute("""
#     SELECT * FROM laundry_detergents
# """)

# _________________________
# end queries
# _________________________

c.execute
rows = c.fetchall()

conn.commit()

conn.close()

# check if any rows are not laundry detergent by seeing if the word in product title.
# for row in rows:
#     if 'Detergent' not in row[0]:
#         print(row[0])

for row in rows:
    print(row)

(2, 'All Mighty Pacs Free Clear Laundry Detergent Pacs ')
(2, 'ECOS Plastic')
(2, 'Seventh Generation Liquid Laundry Detergent ')
(2, 'Tide High Efficiency Liquid Laundry Detergent ')
(3, 'Tide Pods Laundry Detergent Pacs ')
(3, 'Zum Laundry Soap ')


/Users/annaweeks/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,
